## Read .ghg metadata

In [76]:
import preProcessing
import importlib
import time
importlib.reload(preProcessing)
T1 = time.time()

# pr = preProcessing.read_ALL('BB',2023,processes=1,Test=48,reset=1)
pr = preProcessing.read_ALL('BB',2023,processes=4,reset=1)
T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60,' minutes')


2023-03-01 00:00:00
272.4150548665461
Preprocessing Files[████████████████████████████████████████████████████████████] 1318/1318

Runtime:  3.0305892904599507  minutes


In [74]:
import pandas as pd
df = pd.read_csv('C:/highfreq/BB/2023-03-17T003000_LI-7200/2023-03-17T003000_LI-7200.data',skiprows=7,sep='\t')


# df[['flow_rate_7200','signal_strength_7200','signal_strength_7700']]



df.columns[15]


'Total Pressure (kPa)'

In [10]:
df = pd.read_csv('C:/highfreq/BB/2023-03-28T120000_LI-7200/2023-03-28T120000_LI-7200.data',skiprows=7,delimiter='\t')
df.columns

Index(['DATAH', 'Seconds', 'Nanoseconds', 'Sequence Number',
       'Diagnostic Value', 'Diagnostic Value 2', 'Date', 'Time',
       'CO2 Absorptance', 'H2O Absorptance', 'CO2 (mmol/m^3)', 'CO2 (mg/m^3)',
       'H2O (mmol/m^3)', 'H2O (g/m^3)', 'Block Temperature (C)',
       'Total Pressure (kPa)', 'Box Pressure (kPa)', 'Head Pressure (kPa)',
       'Aux 1 - U (m/s)', 'Aux 2 - V (m/s)', 'Aux 3 - W (m/s)',
       'Aux 4 - SOS (m/s)', 'Cooler Voltage (V)', 'Chopper Cooler Voltage (V)',
       'Vin SmartFlux (V)', 'CO2 (umol/mol)', 'CO2 dry(umol/mol)',
       'H2O (mmol/mol)', 'H2O dry(mmol/mol)', 'Dew Point (C)',
       'Cell Temperature (C)', 'Temperature In (C)', 'Temperature Out (C)',
       'Average Signal Strength', 'CO2 Signal Strength', 'H2O Signal Strength',
       'Delta Signal Strength', 'Flow Rate (slpm)', 'Flow Rate (lpm)',
       'Flow Pressure (kPa)', 'Flow Power (V)', 'Flow Drive (%)', 'H2O Sample',
       'H2O Reference', 'CO2 Sample', 'CO2 Reference', 'HIT Power (W)',
 

In [80]:
import setupEP as eP
import importlib
import time
importlib.reload(eP)

########## Note - incomplete ghg files crash program


T1 = time.time()
mR = eP.makeRun('ep_Templates/DefaultSettings.eddypro','BB')
mR.runDates(['2023-03-17 13:00','2023-03-17 16:00'],'test',threads=1,priority = 'high')
T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60)


4
last_change_date
str(datetime.now()).replace(' ','T').split('.')[0]
file_name
file_name
project_title
project_id
project_id
project_id
use_pfile
'1'
proj_file
batch_path+Metadata_File
master_sonic
'csat3_1'
col_air_p
'0'
col_air_t
'0'
col_ts
'0'
pr_subset
'1'
pr_start_date
pr_start_date
pr_start_time
pr_start_time
pr_end_date
pr_end_date
pr_end_time
pr_end_time
out_path
output_path
use_biom
'2'
biom_file
self.sub(self.ini['Paths']['biomet']+self.ini['filenames']['biomet'])
biom_ext
'.'+self.ini['filenames']['biomet'].split('.')[-1]
out_mean_cosp
'1'
out_biomet
'1'
out_mean_spec
'1'
sa_start_date
pr_start_date
sa_end_date
pr_end_date
sa_start_time
pr_start_time
sa_end_time
pr_end_time
data_path
self.sub(self.ini['Paths']['raw'])
pf_start_date
pr_start_date
pf_end_date
pr_end_date
pf_start_time
pr_start_time
pf_end_time
pr_end_time
to_start_date
pr_start_date
to_end_date
pr_end_date
to_start_time
pr_start_time
to_end_time
pr_end_time
3
last_change_date
str(datetime.now()).replace(' ','

In [14]:
import pandas as pd
dateRange = ['2023-03-01 12:00','2023-03-01 13:00']
pd.date_range(dateRange[0],dateRange[1],freq='30T')

DatetimeIndex(['2023-03-01 12:00:00', '2023-03-01 12:30:00',
               '2023-03-01 13:00:00'],
              dtype='datetime64[ns]', freq='30T')

# Speedtests

All times in minutes

### 4 threads (8 timesteps each)

Normal priority: 2.05
High priority: 1.91

### 4 threads (48 timesteps each x 12 days)

High priority: 29

### Eddypro GUI (1 run [12 days, 576 timesteps])

High priority: 90